# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('/Users/monicabustillos/Documents/python-api-challenge/WeatherPy/cityinfo.csv') 
df

,Unnamed: 0,Unnamed: 0.1,City,Latitude,Longitude,Wind Speed,Country,Humidity,Cloudiness,Temperature - F,Max Temperature - F
0,0,0,broken hill,-31.9500,141.4333,3.60,AU,16,0,95.000,95.000
1,1,1,sao filipe,14.8961,-24.4956,3.34,CV,75,44,70.232,70.232
2,2,2,portland,45.5234,-122.6762,1.54,US,69,1,33.548,35.996
3,3,3,kaitangata,-46.2817,169.8464,1.79,NZ,66,100,60.008,60.008
4,4,4,avarua,-21.2078,-159.7750,2.06,CK,78,20,82.400,82.400
...,...,...,...,...,...,...,...,...,...,...,...
577,577,577,kholmogory,64.2229,41.6560,1.00,RU,76,20,-18.400,-18.400
578,578,578,veraval,20.9000,70.3667,1.52,IN,27,83,80.600,80.600
579,579,579,umtata,-31.5889,28.7844,1.54,ZA,94,75,68.000,68.000
580,580,580,magdagachi,53.4500,125.8000,1.12,RU,93,74,9.842,9.842


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = df[['Latitude','Longitude']].astype(float)
humidity = df['Humidity'].astype(float)

In [17]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [55]:
# conditions 
data = df.loc[df['Max Temperature - F']>50,:]
data = df.loc[df['Country']!='US',:]
data = df.loc[df['Wind Speed']<10,:]
data = df.loc[df['Humidity']<75,:]
data

,Unnamed: 0,Unnamed: 0.1,City,Latitude,Longitude,Wind Speed,Country,Humidity,Cloudiness,Temperature - F,Max Temperature - F
0,0,0,broken hill,-31.9500,141.4333,3.60,AU,16,0,95.000,95.000
2,2,2,portland,45.5234,-122.6762,1.54,US,69,1,33.548,35.996
3,3,3,kaitangata,-46.2817,169.8464,1.79,NZ,66,100,60.008,60.008
5,5,5,rikitea,-23.1203,-134.9692,4.29,PF,73,73,78.728,78.728
8,8,8,ushuaia,-54.8000,-68.3000,1.54,AR,72,0,62.600,62.600
...,...,...,...,...,...,...,...,...,...,...,...
568,568,568,sumenep,-7.0167,113.8667,0.35,ID,73,96,83.948,83.948
569,569,569,le port,-20.9373,55.2919,5.66,RE,61,20,77.702,84.200
575,575,575,thai nguyen,21.5928,105.8442,3.60,VN,60,0,69.800,69.800
576,576,576,dabwali,29.9500,74.7333,1.41,IN,20,0,78.242,78.242


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [56]:
hotel_df = data 
hotel_df['Hotel Name'] = ''
hotel_df

/Users/monicabustillos/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,Unnamed: 0.1,City,Latitude,Longitude,Wind Speed,Country,Humidity,Cloudiness,Temperature - F,Max Temperature - F,Hotel Name
0,0,0,broken hill,-31.9500,141.4333,3.60,AU,16,0,95.000,95.000,
2,2,2,portland,45.5234,-122.6762,1.54,US,69,1,33.548,35.996,
3,3,3,kaitangata,-46.2817,169.8464,1.79,NZ,66,100,60.008,60.008,
5,5,5,rikitea,-23.1203,-134.9692,4.29,PF,73,73,78.728,78.728,
8,8,8,ushuaia,-54.8000,-68.3000,1.54,AR,72,0,62.600,62.600,
...,...,...,...,...,...,...,...,...,...,...,...,...
568,568,568,sumenep,-7.0167,113.8667,0.35,ID,73,96,83.948,83.948,
569,569,569,le port,-20.9373,55.2919,5.66,RE,61,20,77.702,84.200,
575,575,575,thai nguyen,21.5928,105.8442,3.60,VN,60,0,69.800,69.800,
576,576,576,dabwali,29.9500,74.7333,1.41,IN,20,0,78.242,78.242,


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [58]:
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    params['location'] = f'{lat},{lon}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    try:
        hotel_df.loc[index,'Hotel Name'] = name_address['results'][0]['name'] 
    except (KeyError, IndexError):
        print('Missing info, skipping')

/Users/monicabustillos/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing info, skipping
Missing info, skipping


In [59]:
hotel_df

,Unnamed: 0,Unnamed: 0.1,City,Latitude,Longitude,Wind Speed,Country,Humidity,Cloudiness,Temperature - F,Max Temperature - F,Hotel Name
0,0,0,broken hill,-31.9500,141.4333,3.60,AU,16,0,95.000,95.000,Broken Hill
2,2,2,portland,45.5234,-122.6762,1.54,US,69,1,33.548,35.996,Portland
3,3,3,kaitangata,-46.2817,169.8464,1.79,NZ,66,100,60.008,60.008,Kaitangata
5,5,5,rikitea,-23.1203,-134.9692,4.29,PF,73,73,78.728,78.728,Rikitea
8,8,8,ushuaia,-54.8000,-68.3000,1.54,AR,72,0,62.600,62.600,Ushuaia
...,...,...,...,...,...,...,...,...,...,...,...,...
568,568,568,sumenep,-7.0167,113.8667,0.35,ID,73,96,83.948,83.948,Sumenep
569,569,569,le port,-20.9373,55.2919,5.66,RE,61,20,77.702,84.200,Le Port
575,575,575,thai nguyen,21.5928,105.8442,3.60,VN,60,0,69.800,69.800,Thái Nguyên
576,576,576,dabwali,29.9500,74.7333,1.41,IN,20,0,78.242,78.242,Mandi Dabwali


In [62]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hover_text=(name_address['results'][0]['name']))
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))